In [17]:
import cv2
import albumentations as A
import os

In [2]:
# Folders
image_dir = "train_images"
mask_dir = "train_masks_fixed"
aug_image_dir = "aug_images"
aug_mask_dir = "aug_masks"

In [9]:
# Create output dirs
os.makedirs(aug_image_dir, exist_ok=True)
os.makedirs(aug_mask_dir, exist_ok=True)

In [4]:
# Define augmentation pipeline
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.3),
    A.GaussNoise(p=0.2),
    A.Blur(blur_limit=3, p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Resize(512, 512),
])

C:\Users\kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [10]:
# Loop through images and masks
for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        img_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename.replace(".jpg", ".png"))  # mask has same name but .png

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # mask is grayscale

        # Generate N augmentations per image
        for i in range(5):   # <--- 5x augmentations
            augmented = train_transform(image=image, mask=mask)
            aug_img = augmented['image']
            aug_mask = augmented['mask']

            # Save with new names
            aug_img_name = f"{os.path.splitext(filename)[0]}_aug{i}.jpg"
            aug_mask_name = f"{os.path.splitext(filename)[0]}_aug{i}.png"

            cv2.imwrite(os.path.join(aug_image_dir, aug_img_name), aug_img)
            cv2.imwrite(os.path.join(aug_mask_dir, aug_mask_name), aug_mask)

print("Augmentation done! All images saved in aug_images/ and aug_masks/")

Augmentation done! All images saved in aug_images/ and aug_masks/


In [3]:
import os
import random
import shutil

# Paths
image_dir = "aug_images"
mask_dir = "aug_masks"
val_image_dir = "val_images"
val_mask_dir = "val_masks"

# Make val dirs
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_mask_dir, exist_ok=True)

# Get all image names (assuming filenames match between images & masks)
all_images = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
print("Total images:", len(all_images))

# Pick 1000 random samples
val_samples = random.sample(all_images, 1000)

# Move or copy them to val set
for img in val_samples:
    mask_name = img.replace(".jpg", ".png")  # adjust if naming is different
    
    shutil.copy(os.path.join(image_dir, img), os.path.join(val_image_dir, img))
    shutil.copy(os.path.join(mask_dir, mask_name), os.path.join(val_mask_dir, mask_name))

print("Validation set created with", len(val_samples), "samples")

Total images: 25438
Validation set created with 1000 samples


In [5]:
import os

val_images = sorted([f for f in os.listdir("val_images") if f.endswith(".jpg")])
val_masks = sorted([f for f in os.listdir("val_masks") if f.endswith(".png")])

print(len(val_images), len(val_masks))

1002 1000


In [6]:
img_names = set([os.path.splitext(f)[0] for f in val_images])
mask_names = set([os.path.splitext(f)[0] for f in val_masks])

print("Images without masks:", img_names - mask_names)
print("Masks without images:", mask_names - img_names)

Images without masks: {'c9f382c7349e_01_aug3', 'acb0fd30b83d_01_aug3'}
Masks without images: set()


In [9]:
import numpy as np
from PIL import Image

def convert_mask(mask_path, save_path):
    mask = np.array(Image.open(mask_path).convert("RGB"))

    # Everything that is NOT white becomes 1
    binary_mask = np.any(mask != [255, 255, 255], axis=-1).astype(np.uint8)

    # Save as 0 (black) and 255 (white)
    Image.fromarray(binary_mask * 255).save(save_path)

In [13]:
import os
# Paths
input_mask_dir = "cars/masks"       # your downloaded dataset masks
output_mask_dir = "cars/new_masks"

# Make sure output folder exists
os.makedirs(output_mask_dir, exist_ok=True)

for filename in os.listdir(input_mask_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        in_path = os.path.join(input_mask_dir, filename)
        out_path = os.path.join(output_mask_dir, filename)
        convert_mask(in_path, out_path)

print("✅ Conversion finished! Binary masks saved.")

✅ Conversion finished! Binary masks saved.


In [20]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.3),
    A.HueSaturationValue(p=0.3),
    
    A.GaussNoise(p=0.2),
    A.Blur(blur_limit=3, p=0.2),
    A.ElasticTransform(p=0.1),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
    
    A.Resize(512, 512),
], additional_targets={'mask': 'mask'})  # ensures mask handled correctly

C:\Users\kumar\AppData\Local\Temp\ipykernel_22300\36687930.py:14: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),


In [23]:
# Folders
image_dir = "cars/img"
mask_dir = "cars/new_masks"
aug_image_dir = "cars/aug_images_new"
aug_mask_dir = "cars/aug_masks_new"

In [24]:
os.makedirs(aug_image_dir, exist_ok=True)
os.makedirs(aug_mask_dir, exist_ok=True)

In [25]:
# Loop through images and masks
for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        img_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename.replace(".jpg", ".png"))  # mask has same name but .png

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # mask is grayscale

        # Generate N augmentations per image
        for i in range(30):   # <--- 5x augmentations
            augmented = train_transform(image=image, mask=mask)
            aug_img = augmented['image']
            aug_mask = augmented['mask']

            # Save with new names
            aug_img_name = f"{os.path.splitext(filename)[0]}_aug{i}.jpg"
            aug_mask_name = f"{os.path.splitext(filename)[0]}_aug{i}.png"

            cv2.imwrite(os.path.join(aug_image_dir, aug_img_name), aug_img)
            cv2.imwrite(os.path.join(aug_mask_dir, aug_mask_name), aug_mask)

print("Augmentation done! All images saved in aug_images/ and aug_masks/")

Augmentation done! All images saved in aug_images/ and aug_masks/


In [27]:
import os
import random
import shutil

# Paths
train_images_dir = "aug_images"
train_masks_dir = "aug_masks"

val_images_dir = "val_images"
val_masks_dir = "val_masks"

# Create validation folders if not exist
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_masks_dir, exist_ok=True)

# Create validation folders if not exist
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_masks_dir, exist_ok=True)

# List of all images (only jpg/jpeg/png)
all_images = [f for f in os.listdir(train_images_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

# Shuffle and select 1000 for validation
random.shuffle(all_images)
val_images = all_images[:1000]

for img in val_images:
    base_name, _ = os.path.splitext(img)   # remove extension
    
    # image src/dst
    img_src = os.path.join(train_images_dir, img)
    img_dst = os.path.join(val_images_dir, img)
    
    # mask (always .png)
    mask_name = f"{base_name}.png"
    mask_src = os.path.join(train_masks_dir, mask_name)
    mask_dst = os.path.join(val_masks_dir, mask_name)
    
    # move
    shutil.move(img_src, img_dst)
    shutil.move(mask_src, mask_dst)

print(f"✅ Moved {len(val_images)} images and masks to validation set")

✅ Moved 1000 images and masks to validation set


In [32]:
val_images = sorted([f for f in os.listdir("aug_images") if f.endswith(".jpg")])
val_masks = sorted([f for f in os.listdir("aug_masks") if f.endswith(".png")])

print(len(val_images), len(val_masks))

25037 25040


In [33]:
img_names = set([os.path.splitext(f)[0] for f in val_images])
mask_names = set([os.path.splitext(f)[0] for f in val_masks])

print("Images without masks:", img_names - mask_names)
print("Masks without images:", mask_names - img_names)

Images without masks: set()
Masks without images: {'acb0fd30b83d_01_aug3', '54ee69d7d5e5_04_aug2', 'c9f382c7349e_01_aug3'}
